In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import fastai

from fastai.io import *

from fastai.column_data import *
from fastai.text import *


from pathlib import Path

/home/nrhodes/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/home/nrhodes/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Learning to generate Dickens-like text

### Setup

In [2]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

PATH=Path('../data/dickens/')

TRN_PATH = 'trn'
VAL_PATH = 'val'
TRN = PATH / TRN_PATH
VAL = PATH / VAL_PATH

%ls {PATH}

davidcopperfield.txt  models/  tmp/  trn/  val/


In [3]:
TEXT = data.Field(lower=True, tokenize=list)
bs=2048; bptt=16; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(44, 53, 1, 1494913)

### Our model

In [4]:
class CharSequence(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.dropout = nn.Dropout(.1)
        self.rnn = nn.GRU(n_fac, n_hidden, nl, dropout=0.0)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.dropout(self.e(cs)), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = V(torch.zeros(self.nl, bs, n_hidden))

In [5]:
char = CharSequence(md.nt, n_fac, n_hidden, 1)
if torch.cuda.is_available():
    char = char.cuda()
m = BasicModel(char)
learner = RNN_Learner(md, m, opt_fn=optim.Adam, crit=F.nll_loss)

#minimum_learning_rate_divisor = 1200
#percent_after_triangle_cycle = 15
#max_momentum=.97
#min_momentum=.85
#learner.fit(1e-2, 1, cycle_len=72, 
#           use_clr_beta=(minimum_learning_rate_divisor, 
#                         percent_after_triangle_cycle, 
#                         max_momentum, 
#                         min_momentum),
#                         wds=1e-5)
learner.fit(2e-2, 7, cycle_mult=2, cycle_len=1, 
           wds=1e-5)


epoch      trn_loss   val_loss   
    0      2.422565   2.140659  
    1      2.0522     1.785665  
    2      1.86702    1.733811  
    3      1.777272   1.680565  
    4      1.706417   1.62905   
    5      1.652503   1.621268  
    6      1.615807   1.609822  
    7      1.62899    1.64069   
    8      1.624412   1.640711  
    9      1.609931   1.576252  
    10     1.58831    1.577713  
    11     1.566153   1.559515  
    12     1.536029   1.556644  
    13     1.516173   1.535826  
    14     1.499759   1.513017  
    15     1.588607   1.658486  
    16     1.594105   1.588101  
    17     1.600745   1.583522  
    18     1.585102   1.598893  
    19     1.58912    1.595601  
    20     1.568464   1.552763  
    21     1.565605   1.565839  
    22     1.541029   1.5718    
    23     1.515897   1.523929  
    24     1.515345   1.523382  
    25     1.505235   1.520322  
    26     1.484016   1.52308   
    27     1.481771   1.507402  
    28     1.471043   1.505026  
    29   

[array([1.4716])]

### Test

In [6]:
def get_next(inp):
    device = None if torch.cuda.is_available() else -1
    idxs = TEXT.numericalize(inp, device=device, train=False)
    p = learner.model(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [7]:
get_next('for thos')
#TEXT

'e'

In [8]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [9]:
print(get_next_n('fourscore and seven years ago', 400))

fourscore and seven years agony of the rooks with me in the adverting. i was certain by further thanour houked.‘mutt--that i wasgo by the sy-point. they seethink of my son;but topity that she desired in a handkarch. ‘--change enough to murdstone. i really remember thatshe had one of me, and that tened them atmany napid scarrierydo?” yet find a frivly; and then the solicitof ‘having, and if mr. quinion traddles, ‘ifeversible h
